In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.similarities.docsim import Similarity
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from gensim.models import Word2Vec, Doc2Vec
from bs4 import BeautifulSoup
import re
import nltk.data
import string
from gensim.models.doc2vec import TaggedDocument
from gensim import matutils
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\admin\Anaconda2\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 650 Ti (CNMeM is disabled, cuDNN not available)


In [2]:
iteminfo_train = pd.read_csv('ItemInfo_train.csv')
#iteminfo_test = pd.read_csv('ItemInfo_test.csv')

In [3]:
itempairs_train = pd.read_csv('ItemPairs_train.csv')
#itempairs_test = pd.read_csv('ItemPairs_test.csv')

In [4]:
location = pd.read_csv('Location.csv')
category = pd.read_csv('Category.csv')

In [5]:
iteminfo_train_loc = pd.merge(pd.merge(iteminfo_train, location, how = 'left', on = 'locationID'), category, how = 'left', on = 'categoryID')
#iteminfo_test_loc = pd.merge(pd.merge(iteminfo_test, location, how = 'left', on = 'locationID'), category, how = 'left', on = 'categoryID')

In [6]:
x = pd.merge(pd.merge(itempairs_train, iteminfo_train_loc, how = 'left', left_on = 'itemID_1', right_on = 'itemID'), 
                    iteminfo_train_loc, how = 'left', left_on = 'itemID_2', right_on = 'itemID')
#x_test = pd.merge(pd.merge(itempairs_test, iteminfo_test_loc, how = 'left', left_on = 'itemID_1', right_on = 'itemID'), 
#                    iteminfo_test_loc, how = 'left', left_on = 'itemID_2', right_on = 'itemID')

In [7]:
x['title_x'] = x.title_x.map(lambda x : '' if pd.isnull(x) else x)
x['title_y'] = x.title_y.map(lambda x : '' if pd.isnull(x) else x)
x['description_x'] = x.description_x.map(lambda x : '' if pd.isnull(x) else x)
x['description_y'] = x.description_y.map(lambda x : '' if pd.isnull(x) else x)
#x_test['title_x'] = x_test.title_x.map(lambda x : '' if pd.isnull(x) else x)
#x_test['title_y'] = x_test.title_y.map(lambda x : '' if pd.isnull(x) else x)
#x_test['description_x'] = x_test.description_x.map(lambda x : '' if pd.isnull(x) else x)
#x_test['description_y'] = x_test.description_y.map(lambda x : '' if pd.isnull(x) else x)

In [8]:
def clean_text(document):    
    # Removes punctuations
    document = document.translate(None, string.punctuation)
    
    # Change to lowercase
    document = document.lower()
    
    # Removes newline characters
    document = ' '.join(document.split('\n'))
    
    # Replace multiple spaces with one space
    document = ' '.join(document.split())
    #document = nlp(unicode(document))
    
    return document

def word_vec(x, model, num_features):
    try:
        a = model[x]
        b = 1
    except KeyError:
        a = np.zeros(num_features)
        b = 0
    return [a,b]

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    
    for word in words:
        a, b = word_vec(word, model, num_features)
        nwords = nwords + b
        featureVec = np.add(featureVec, a)
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [9]:
# Word2Vec
sentences_desc_x = []
sentences_desc_y = []
sentences_title_x = []
sentences_title_y = []
for i in range(len(x.description_x)):
    if i in [10, 500000, 1000000, 1500000, 2000000]:
        print i
    sentences_desc_x.append(clean_text(x.description_x[i]).decode('utf-8').split())
    sentences_desc_y.append(clean_text(x.description_y[i]).decode('utf-8').split())
for i in range(len(x.title_x)):
    if i in [10, 500000, 1000000, 1500000, 2000000]:
        print i
    sentences_title_x.append(clean_text(x.title_x[i]).decode('utf-8').split())
    sentences_title_y.append(clean_text(x.title_y[i]).decode('utf-8').split())

10
500000
1000000
1500000
2000000
10
500000
1000000
1500000
2000000


In [10]:
kf = KFold(n_splits = 5, random_state = 1)
train_score = []
test_score = []
log_reg = LogisticRegression()
for train_index, test_index in kf.split(x):
    sentences_desc_x_train = np.array(sentences_desc_x)[train_index].tolist()
    sentences_desc_y_train = np.array(sentences_desc_y)[train_index].tolist()
    sentences_desc_x_test = np.array(sentences_desc_x)[test_index].tolist()
    sentences_desc_y_test = np.array(sentences_desc_y)[test_index].tolist()
    sentences_title_x_train = np.array(sentences_title_x)[train_index].tolist()
    sentences_title_y_train = np.array(sentences_title_y)[train_index].tolist()
    sentences_title_x_test = np.array(sentences_title_x)[test_index].tolist()
    sentences_title_y_test = np.array(sentences_title_y)[test_index].tolist()
    print 'w2v_x'
    model_desc_x = Word2Vec(sentences_desc_x_train, min_count = 1, size = 50)
    print 'w2v_y'
    model_desc_y = Word2Vec(sentences_desc_y_train, min_count = 1, size = 50)
    print 'w2v_x'
    model_title_x = Word2Vec(sentences_title_x_train, min_count = 1, size = 50)
    print 'w2v_y'
    model_title_y = Word2Vec(sentences_title_y_train, min_count = 1, size = 50)
    print 'cos_sim_x'
    x_train, x_test = x.loc[train_index, :], x.loc[test_index, :]
    cos_sim_desc_train = []
    cos_sim_desc_test = []
    num_features = model_desc_x.wv.syn0.shape[1]
    for i in range(len(sentences_desc_x_train)):
        cos_sim_desc_train.append(cosine_similarity(np.array([makeFeatureVec(sentences_desc_x_train[i], model_desc_x, num_features),
                                                        makeFeatureVec(sentences_desc_y_train[i], model_desc_y, num_features)]))[1, 0])

    for i in range(len(sentences_desc_x_test)):
        cos_sim_desc_test.append(cosine_similarity(np.array([makeFeatureVec(sentences_desc_x_test[i], model_desc_x, num_features),
                                                        makeFeatureVec(sentences_desc_y_test[i], model_desc_y, num_features)]))[1, 0])
    
    cos_sim_title_train = []
    cos_sim_title_test = []
    num_features = model_desc_x.wv.syn0.shape[1]
    for i in range(len(sentences_title_x_train)):
        cos_sim_title_train.append(cosine_similarity(np.array([makeFeatureVec(sentences_title_x_train[i], model_title_x, num_features),
                                                        makeFeatureVec(sentences_title_y_train[i], model_title_y, num_features)]))[1, 0])

    for i in range(len(sentences_desc_x_test)):
        cos_sim_title_test.append(cosine_similarity(np.array([makeFeatureVec(sentences_title_x_test[i], model_title_x, num_features),
                                                        makeFeatureVec(sentences_title_y_test[i], model_title_y, num_features)]))[1, 0])
    
    log_reg.fit(pd.DataFrame(np.column_stack([cos_sim_desc_train, cos_sim_title_train])), np.array(x_train.isDuplicate))
    pred_train = log_reg.predict(pd.DataFrame(np.column_stack([cos_sim_desc_train, cos_sim_title_train])))
    pred_test = log_reg.predict(pd.DataFrame(np.column_stack([cos_sim_desc_test, cos_sim_title_test])))
    train_score.append(pred_train)
    test_score.append(pred_test)
    print "Test Accuracy:", accuracy_score(x_test.isDuplicate, pred_test)
    print "Train Accuracy:", accuracy_score(x_train.isDuplicate, pred_train)

w2v_x
w2v_y
w2v_x
w2v_y
cos_sim_x
Test Accuracy: 0.608407434646
Train Accuracy: 0.60927426836
w2v_x
w2v_y
w2v_x
w2v_y
cos_sim_x
Test Accuracy: 0.601832924104
Train Accuracy: 0.601615800648
w2v_x
w2v_y
w2v_x
w2v_y
cos_sim_x
Test Accuracy: 0.607101369094
Train Accuracy: 0.607410753423
w2v_x
w2v_y
w2v_x
w2v_y
cos_sim_x
Test Accuracy: 0.603813605358
Train Accuracy: 0.605709206863
w2v_x
w2v_y
w2v_x
w2v_y
cos_sim_x
Test Accuracy: 0.610148442449
Train Accuracy: 0.611352474618
